# Training Yolo with MS Coco 

In [1]:
# all imports
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, os.path.abspath('./src'))


from torchvision.datasets import CocoDetection
from torchvision import transforms

from datasets.preprocess import *
from datasets.YaYoloCocoDataset import YaYoloCocoDataset

from yolo.yolo import Yolo
import torch
print(torch.cuda.is_available())



[2019-11-15 12:25:06,612] {device:11} INFO - ************ Using cpu as device **************


False


In [3]:
cfg_file = './cfg/yolov3.cfg'
weight_file = './cfg/yolov3.weights'
batch_size = 32
COCO_IMAGES_DIR = '/home/peter/datasets/coco-small/cocoapi/images/train2014'
#COCO_IMAGES_DIR = '/home/ubuntu/datasets/coco/train2014'
COCO_ANNOTATIONS_FILE = '/home/peter/datasets/coco-small/cocoapi/annotations/instances_train2014_10_per_category.json'
#COCO_ANNOTATIONS_FILE = '/home/ubuntu/datasets/coco/annotations/instances_train2014.json'



image_and_target_transform = Compose([
        SquashResize(416),
        ConvertXandYToCenterOfBoundingBox(),
        ScaleBboxRelativeToSize(416),
        # PadToFit(255),
        # RandomCrop(200),
        # RandomHorizontalFlip(),,
        CocoToTensor()
    ])

ya_yolo_dataset = YaYoloCocoDataset(images_dir=COCO_IMAGES_DIR, 
                                    annotations_file=COCO_ANNOTATIONS_FILE,
                                    transforms=image_and_target_transform,
                                    batch_size=batch_size)
model = Yolo(cfg_file=cfg_file, batch_size=batch_size)
model.load_weights(weight_file)


loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
Loading weights. Please Wait...



In [ ]:
from yolo.training import training

training(model=model, ya_yolo_dataset=ya_yolo_dataset, model_dir='./models', num_epochs=2, limit=None, debug=False, print_every=10)

Number of images:  82783
Epoch: 1, Batch: 1, Avg. Loss: 1.526620239019394e-05
Epoch: 1, Batch: 11, Avg. Loss: 0.007035233966521023
Epoch: 1, Batch: 21, Avg. Loss: 0.006873827660610834
Epoch: 1, Batch: 31, Avg. Loss: 0.052673455193334176
Epoch: 1, Batch: 41, Avg. Loss: 0.00795486084165608
Epoch: 1, Batch: 51, Avg. Loss: 0.006171065682160934
Epoch: 1, Batch: 61, Avg. Loss: 0.007423163860176735
Epoch: 1, Batch: 71, Avg. Loss: 0.011372890894146911
